# Import and sandbox

In [1]:
import numpy as np
import pandas as pd
import torch 
from tqdm import tqdm
import matplotlib.pyplot as plt


# check n_wells 

## Imports

In [2]:
df_phenom = pd.read_parquet('/projects/synsight/data/jump_embeddings/wells_embeddings/openphenom/metadata_openphenom.parquet')

In [2]:
df_phenom = pd.read_parquet('/home/maxime/data/jump_embeddings/metadata_dinov2_g.parquet')


In [3]:
df_phenom.drop(columns=['Metadata_Well', 'Metadata_JCP2022', 'Metadata_InChI'], inplace=True)

In [4]:
df_phenom = df_phenom.drop_duplicates()

In [5]:
min_samples_per_source = df_phenom.groupby('Metadata_Source').size().min()

balanced_sample = df_phenom.groupby('Metadata_Source').apply(
    lambda group: group.groupby('Metadata_Batch').apply(
        lambda batch: batch.sample(n=min(len(batch), min_samples_per_source // len(group['Metadata_Batch'].unique())))
    ).reset_index(drop=True)
).reset_index(drop=True)



/tmp/ipykernel_1173591/2387238680.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  lambda group: group.groupby('Metadata_Batch').apply(
/tmp/ipykernel_1173591/2387238680.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  lambda group: group.groupby('Metadata_Batch').apply(
/tmp/ipykernel_1173591/2387238680.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior i

In [6]:
balanced_sample['Metadata_Source'].value_counts()


Metadata_Source
source_10    51
source_11    50
source_8     48
source_1     45
source_9     44
source_3     39
source_2     39
source_6     39
source_5     33
Name: count, dtype: int64

In [15]:
import pandas as pd

# Calculer le nombre maximum de plaques échantillonnées uniformément
min_samples_per_source = df_phenom.groupby('Metadata_Source').size().min()

# Échantillonner un nombre fixe de plaques par source
balanced_sample = df_phenom.groupby('Metadata_Source').apply(
    lambda group: group.sample(n=min_samples_per_source, random_state=42)  # Fixer un seed pour reproductibilité
).reset_index(drop=True)

balanced_sample['Metadata_Source'].value_counts()

/tmp/ipykernel_1173591/1876179003.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  balanced_sample = df_phenom.groupby('Metadata_Source').apply(


Metadata_Source
source_1     51
source_10    51
source_11    51
source_2     51
source_3     51
source_5     51
source_6     51
source_8     51
source_9     51
Name: count, dtype: int64

In [21]:
import pandas as pd

# Calculate the number of plates per source to maintain balance
n_sources = df_phenom['Metadata_Source'].nunique()

# Ensure a minimum number of plates per source if possible
def balanced_sample_with_total_limit(df, total_limit=48):
    n_sources = df['Metadata_Source'].nunique()
    plates_per_source = total_limit // n_sources

    # First, sample equally across sources
    sampled = (
        df.groupby('Metadata_Source', group_keys=False).apply(
            lambda source_group: source_group.sample(
                n=min(len(source_group), plates_per_source), random_state=42)
        )
    )

    # If we have fewer than the total_limit, fill the remaining slots
    if len(sampled) < total_limit:
        remaining_slots = total_limit - len(sampled)
        remaining_sample = df.loc[~df.index.isin(sampled.index)].sample(n=remaining_slots, random_state=42)
        sampled = pd.concat([sampled, remaining_sample])

    return sampled

# Apply the function
balanced_sample = balanced_sample_with_total_limit(df_phenom, total_limit=48)


/tmp/ipykernel_1173591/1415224989.py:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df.groupby('Metadata_Source', group_keys=False).apply(


In [22]:
balanced_sample['Metadata_Source'].value_counts()

Metadata_Source
source_9     7
source_11    6
source_1     5
source_2     5
source_10    5
source_3     5
source_5     5
source_6     5
source_8     5
Name: count, dtype: int64

In [24]:
balanced_sample.info()

<class 'pandas.core.frame.DataFrame'>
Index: 48 entries, 199014 to 131073
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Metadata_Source  48 non-null     object
 1   Metadata_Batch   48 non-null     object
 2   Metadata_Plate   48 non-null     object
dtypes: object(3)
memory usage: 1.5+ KB


# Export and save

In [25]:
balanced_sample.to_json("balanced_plates.json", orient="records")
